# ketos jupyter notebook machine learning example

this is an example for a ketos jupyter notebook.

to deploy the notebook in order to allow for external predictions just do the following:
1. tag the cells that you would like to be executed for your prediction with the keyword "ketos_predict" in the first line of each cell as a comment.

2. tag the cells which produce the output for the prediction with the keyword "ketos_predict_output", please note that this cell will be executed and the first line of the output will be used as return value for the prediction


In [ ]:
# ketos_init
install.packages(c('caret', 'plyr', 'e1071', 'glmnet'))

In [ ]:
# ketos_train
genPrep = function(){  
  library(caret)
}

splitData = function(seed, data){
# randomly splits data into training (0.8) and test (0.2) data
  
  set.seed(seed)
  training_index = createDataPartition(data[[ncol(data)]], p = 0.8, list = F)
  training = data[training_index,]
  test = data[-training_index,]
  
  return(list(training=training,test=test))
}

In [ ]:
# ketos_train
ketos_train = function(dataset = NULL, outcomeVar = NULL, metric = "Accuracy", control = NULL){
  # trains model
  form = as.formula(paste(c(outcomeVar, "~."), collapse=" "))
  
  print("start train GLMNET")
  set.seed(seed)
  trained_model = caret::train(form, data=dataset, method="glmnet", metric=metric, preProc=c("center", "scale"), trControl=control)

  return(trained_model)
  
}

In [ ]:
# ketos_train
ketos_Input_data = ("paste your training data url from your ketos system here to get your training data")
#ketos_data_file = "ketos_predict_example.csv"

#ketos_data_train = read.csv(ketos_data_file, header = TRUE, sep = ";", quote = "\"",dec = ".", fill = TRUE, comment.char = "")
ketos_data_train = read.csv(url(ketos_Input_data))
genPrep()

seed = 7
#splitData = splitData(seed, ketos_data_train)
#trainingData = splitData$training
#testData = splitData$test
#fileTrain = "training.csv"
#fileTest  = "test.csv"
#write.table(trainingData, fileTrain, sep = ";", quote = TRUE, dec = ".", row.names = FALSE, append = FALSE)
#write.table(testData, fileTest, sep = ";", quote = TRUE, dec = ".", row.names = FALSE, append = FALSE)

control = trainControl(method="repeatedcv", number=10, repeats=10, sampling="down", savePredictions="all")
outcomeVar = "human.body.structure"
ketos_model = ketos_train(ketos_data_train, outcomeVar,"Accuracy",control)
modeltoSave = 'ketos_model'
save(ketos_model,file = modeltoSave)

In [ ]:
#ketos_predict, ketos_predict_output

format_to_json = function(patientIds, predictions){
    ret_json = "["
    ret_json = paste(ret_json,'{"patientId":"',patientIds[1] ,'", "prediction":"',predictions[1], '"}', sep="")
    
    if(!length(patientIds == 1)){
    for(i in 2:length(patientIds)){
        patientId = patientIds[i]
        prediction = predictions[i]
        
        ret_json = paste(ret_json,',{"patientId":"',patientId ,'", "prediction":"' ,prediction, '"}', sep="")
    }
        }
    ret_json = paste(ret_json, "]", sep="")
    
    return (ret_json)
}

ketos_predict = function(){
    ketos_predict_data = ("csv url will be inserted for the predict call here automatically - paste your url here to test ur predict function")
    load("ketos_model")
    predict_data = read.csv(url(ketos_predict_data))
    #predict_data = read.csv('test.csv', header = TRUE, sep = ";", quote = "\"",dec = ".", fill = TRUE, comment.char = "")
    
    prediction = predict(ketos_model, predict_data)
    prediction = format_to_json(predict_data$subject, prediction)
    return (prediction)
}
#print(ketos_predict())
print(ketos_predict(), quote=FALSE)
